In [37]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_goldfeldquandt, het_breuschpagan
from scipy import stats
import statsmodels.stats.stattools as stattools
from statsmodels.compat import lzip

In [17]:
# Generate a toy dataset
n = 100

np.random.seed(42)
X = np.random.rand(n, 2)  # Two features

# Add constant term to X for intercept
X = sm.add_constant(X)

y = np.random.rand(n)  # Target variable

In [18]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Fit a OLS model
model = sm.OLS(y_train, X_train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.7702
Date:                Tue, 07 Nov 2023   Prob (F-statistic):              0.466
Time:                        11:43:44   Log-Likelihood:                -13.736
No. Observations:                  80   AIC:                             33.47
Df Residuals:                      77   BIC:                             40.62
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5823      0.082      7.107      0.000       0.419       0.745
x1             0.0038      0.104      0.037      0.971      -0.204       0.212
x2            -0.1413      0.114     -1.241      0.218      -0.368       0.085
==============================================================================
Omnibus:                       16.841   Durbin-Watson:                   2.255
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                4.282
Skew:                          -0.102   Prob(JB):                        0.118
Kurtosis:                       1.885   Cond. No.                         5.04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
0.0038/0.104

0.03653846153846154

In [21]:
# df = n - k - 1 where k is no of features
n = 80
k = 2
df = n - k - 1
stats.t.cdf(-1.241,df=df)*2

0.2183733703090349

In [22]:
# Ho: residuals are normal
stats.jarque_bera(model.resid)

SignificanceResult(statistic=4.281983597317015, pvalue=0.11753821095362818)

In [23]:
# Perform Durbin-Watson test
dw_value = stattools.durbin_watson(model.resid)

print(dw_value)

2.2548148377375328


In [24]:
# f stat is calculated as (SSR / k) / (SSE / n-k-1)
yhat_train = model.predict(X_train)
ybar_train = np.mean(y_train)
MSR = np.sum((yhat_train - ybar_train)**2)/k
MSE = np.sum((y_train - yhat_train)**2)/(n-k-1)
print("f stat:", MSR / MSE)

f stat: 0.7702113679517493


In [25]:
# pvalue of f stat
dfn = k
dfd = n - k - 1
f_stat = 0.7702
stats.f.sf(f_stat, dfn=dfn, dfd=dfd)

0.46645336626060396

In [26]:
# Perform Goldfeld-Quandt test
name, f_stat, p_value = het_goldfeldquandt(model.resid, model.model.exog)
print(f"Test statistic: {f_stat}")
print(f"p-value: {p_value}")

Test statistic: 0.7524013042332454
p-value: increasing


In [27]:
# Perform Goldfeld-Quandt test
name, f_stat, p_value = het_goldfeldquandt(model.resid, model.model.exog)
print(f"Test statistic: {f_stat}")
print(f"p-value: {p_value}")

Test statistic: 0.7524013042332454
p-value: increasing


In [40]:
# perform Breusch-Pagan test using residuals and independent variables
# compute residuals using 'resid'
# 'exog' returns the independent variables in the model alng with the intercept
test = het_breuschpagan(model.resid, model.model.exog)
test

(1.482131701797389, 0.4766056541389392, 0.7267399352015493, 0.4867688326204046)